In [ ]:
import pandas as pd

# Carregar o dataset
dados = pd.read_csv("Historico_de_materias.csv")

# Exibir as 5 primeiras linhas do dataset
dados.head() 


In [ ]:
#Verificação do número de linhas e colunas
dados.shape

In [ ]:
# Removendo todas as linhas onde o assunto não é "esporte"
dados_esporte = dados[~dados['assunto'].isin(['economia', 'política', 'famosos'])]

In [ ]:
# Exibir as 5 primeiras linhas do novo dataframe 'dados_esporte'
dados_esporte.head()

In [ ]:
#Verificação do número de linhas e colunas
dados_esporte.shape


In [ ]:
# Exibe as colunas para entender a estrutura
print(dados_esporte.columns)


In [ ]:
# Exemplo de leitura de uma notícia

# 'conteudo_noticia' é o nome da coluna que armazena o texto
print(dados_esporte['conteudo_noticia'][74])  

In [ ]:
import re

# Função de limpeza do texto ajustada para lidar com NaN
def limpar_texto(texto):
    if isinstance(texto, str):  # Verifica se o valor é uma string
        texto = texto.lower()  # Normalização para minúsculas
        texto = re.sub(r'[^\w\s]', '', texto)  # Remove pontuações
        texto = re.sub(r'\d+', '', texto)  # Remove números
        return texto
    return ""  # Retorna uma string vazia para valores não textuais (ex: NaN)

# Aplicando a função a toda a coluna 'conteudo_noticia' de uma vez
dados_esporte['conteudo_limpo'] = dados_esporte['conteudo_noticia'].apply(limpar_texto)

# Exibindo o conteúdo limpo da linha 74 para conferir
print(dados_esporte['conteudo_limpo'][74])


In [ ]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('portuguese'))

# Função para remover stopwords
def remover_stopwords(texto):
    palavras = texto.split()
    palavras_filtradas = [palavra for palavra in palavras if palavra not in stop_words]
    return ' '.join(palavras_filtradas)

# Aplicando a função ao dataset
dados_esporte['conteudo_sem_stopwords'] = dados_esporte['conteudo_limpo'].apply(remover_stopwords)
print(dados_esporte['conteudo_sem_stopwords'][74])


In [ ]:
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

# Função para lematização
def lematizar_texto(texto):
    palavras = texto.split()
    palavras_lematizadas = [lemmatizer.lemmatize(palavra) for palavra in palavras]
    return ' '.join(palavras_lematizadas)

# Aplicando a função de lematização
dados_esporte['conteudo_lematizado'] = dados_esporte['conteudo_sem_stopwords'].apply(lematizar_texto)
print(dados_esporte['conteudo_lematizado'].head())


In [ ]:
# Carregar o modelo de linguagem do spaCy
nlp = spacy.load("pt_core_news_sm")


In [ ]:
# Configurar stopwords
api_stop_words = set(stopwords.words('portuguese'))
minhas_stop_words = {'a', 'e', 'i', 'o', 'u'}
stop_words = api_stop_words | minhas_stop_words

In [ ]:
# Configurar stopwords
api_stop_words = set(stopwords.words('portuguese'))
minhas_stop_words = {'a', 'e', 'i', 'o', 'u'}
stop_words = api_stop_words | minhas_stop_words

In [ ]:
# Função para fazer o tratamento de linguagem natural no texto
def tratamento_pln(texto):
    texto = texto.lower()  # Normalizar para minúsculas
    texto = re.sub(r'[^a-zA-Záéíóú\s]', '', texto)  # Remover números e pontuações
    doc = nlp(texto)
    clean_tokens = [token.lemma_ for token in doc if token.text not in stop_words and not token.is_punct]
    clean_text = ' '.join(clean_tokens)
    return clean_text

In [ ]:
# Limpar e tratar o texto das notícias
dados_esporte['conteudo_tratado'] = dados_esporte['conteudo_noticia'].apply(tratamento_pln)

In [ ]:
# Dividir o texto em chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=40, chunk_overlap=10)
chunks = [chunk for noticia in dados['conteudo_tratado'] for chunk in text_splitter.split_text(noticia)]

In [ ]:
# Carregar o modelo de embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(chunks)

In [ ]:
# Gerar IDs para cada chunk
uids = [f"noticia_{i}" for i in range(len(chunks))]

In [ ]:
# Conectar ao banco de dados vetorial ChromaDB
client = chromadb.Client()
collection = client.create_collection(name="noticias_vetorial")

In [ ]:
# Adicionar documentos ao banco de dados
collection.add(documents=chunks, embeddings=embeddings, ids=uids)

In [ ]:
# Função para consulta ao banco vetorial
def consulta_vetorial(query_text):
    query_embedding = model.encode([query_text])
    results = collection.query(query_embeddings=query_embedding, n_results=1)
    for i in range(len(results['ids'][0])):
        doc_id = results['ids'][0][i]
        distance = results['distances'][0][i]
        document = results['documents'][0][i]
        print(f"ID: {doc_id}")
        print(f"Distância: {distance}")
        print(f"Documento: {document}")
        print("-" * 40)

In [ ]:
# Exemplo de consulta
consulta_vetorial("termos de busca")

In [ ]:
# Função para dividir o texto em chunks de N palavras
def dividir_em_chunks(texto, tamanho_chunk=50):
    palavras = texto.split()
    chunks = [' '.join(palavras[i:i + tamanho_chunk]) for i in range(0, len(palavras), tamanho_chunk)]
    return chunks

# Aplicando a divisão no dataset
dados_esporte['chunks'] = dados_esporte['conteudo_lematizado'].apply(dividir_em_chunks)
print(dados_esporte['chunks'][74])


In [ ]:
from transformers import BertModel, BertTokenizer
import torch

# Carregar o modelo e o tokenizer do BERTimbau
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
model = BertModel.from_pretrained('neuralmind/bert-base-portuguese-cased')

# Função para criar embeddings
def criar_embedding(texto):
    inputs = tokenizer(texto, return_tensors='pt', truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze()  # Média das embeddings
    return embeddings.detach().numpy()

# Aplicar embedding em um dos chunks
dados_esporte['embeddings'] = dados_esporte['chunks'].apply(lambda chunks: [criar_embedding(chunk) for chunk in chunks])
print(dados_esporte['embeddings'][74])


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Função para calcular a similaridade de uma nova consulta com o texto
def similaridade_coseno(embedding1, embedding2):
    return cosine_similarity([embedding1], [embedding2])[0][0]

# Exemplo de similaridade entre os embeddings do primeiro e segundo chunks de um texto
similaridade = similaridade_coseno(dados_esporte['embeddings'][0][0], dados_esporte['embeddings'][0][1])
print("Similaridade coseno entre chunks:", similaridade)
